In [48]:
import pandas as pd

In [30]:
df = pd.read_excel("Input.xlsx")
print(df.head())
df.shape
#print(df['URL'][:3])

            URL_ID                                                URL
0  Netclan20241017  https://insights.blackcoffer.com/ai-and-ml-bas...
1  Netclan20241018  https://insights.blackcoffer.com/enhancing-fro...
2  Netclan20241019  https://insights.blackcoffer.com/roas-dashboar...
3  Netclan20241020  https://insights.blackcoffer.com/efficient-pro...
4  Netclan20241021  https://insights.blackcoffer.com/development-o...


(147, 2)

In [49]:
import requests
from bs4 import BeautifulSoup

In [50]:
import os

In [ ]:
# Code for iterating through every url and extracting title and text then saving it to Folder Article.0-147 by URL_ID
def savearticle(article_title, article_text):
    if not (os.path.exists(".\\Article.0-147")):
        os.makedirs(".\\Article.0-147")

    with open(f".\\Article.0-147\\{urlid}.txt","w",encoding="utf-8") as file:
        file.write(article_title+"\n")
        file.write(article_text)

for urlid, url in zip(df['URL_ID'], df['URL']):
    #print(urlid, url)
    response = requests.get(url)
    response.raise_for_status()

    soup = BeautifulSoup(response.content, "html.parser")

    article_title= soup.find("h1", class_ = "entry-title").get_text(strip=True)
    content_div= soup.find("div", class_= "td-post-content tagdiv-type")
    article_text = content_div.get_text(separator="\n", strip=True)

    savearticle(article_title, article_text)



In [5]:
article_folder = ".\\Article.0-147"
stopwords_folder = ".\\StopWords"
master_folder = ".\\MasterDictionary"
filtered_article_folder = ".\\FilteredStopwords" 

article_files = os.listdir(article_folder)
stopwords_files = os.listdir(stopwords_folder)
master_files = os.listdir(master_folder)

In [6]:


stopword_set = set() # create a set to extract all the unique text from all stopword file

def compare_stopwords(words, stopwords_files):
    for sw_file in stopwords_files:
        cleaned_sw = []

        with open(f"{stopwords_folder}\\{sw_file}","r") as sf:
                sw_content = sf.read().split()
                sw_words = [ w.replace("|", " ").strip() for w in sw_content ]
                cleaned_sw = [ w.strip().lower() for w  in sw_words if w.strip() ]
                for x in cleaned_sw:
                     stopword_set.add(x)
                # print(cleaned_sw)

                filtered_words = [ w for w in words if w.lower() not in cleaned_sw]

    return filtered_words



for article in article_files:
    with open(f"{article_folder}\\{article}","r", encoding="utf-8") as art:
        content = art.read().split()
        

        filtered_words = compare_stopwords(content,stopwords_files)
        # print(filtered_words)

        if not (os.path.exists(filtered_article_folder)):
            os.makedirs(filtered_article_folder)
            
        with open(f'{filtered_article_folder}\\{article}','w', encoding='utf-8') as f_sw:
            for word in filtered_words:
                f_sw.write(word+" ")

# print(len(stopword_set))


               

In [7]:
for master in master_files:
     with open(f"{master_folder}\\{master}","r") as f:
          if(master == "positive-words.txt"):
               positive_words = set(word.strip().lower() for word in f if word.strip())
          else:
               negative_words = set(word.strip().lower() for word in f if word.strip())
print(positive_words, negative_words, len(positive_words), len(negative_words),sep="\n")

{'enchant', 'godlike', 'impressed', 'imaculate', 'morality', 'right', 'well-received', 'precisely', 'freed', 'lovably', 'refreshed', 'panoramic', 'praise', 'reasoned', 'encouragement', 'laud', 'felicity', 'irreplaceable', 'high-quality', 'dignity', 'inviolable', 'fecilitous', 'accomplished', 'clearer', 'spellbindingly', 'delicious', 'uplifting', 'unequivocal', 'glitter', 'self-sufficiency', 'enviably', 'gleefully', 'reformed', 'elan', 'sufficient', 'sufficiently', 'brave', 'maturely', 'favour', 'humility', 'abound', 'simpler', 'prestige', 'advantage', 'mature', 'profound', 'lover', 'handsome', 'sensationally', 'suffice', 'terrific', 'graceful', 'captivate', 'sagely', 'evocative', 'coherent', 'proud', 'recovery', 'awesomeness', 'faithfully', 'savings', 'hilarious', 'kindly', 'eventful', 'amazed', 'righteousness', 'propitious', 'affability', 'flexible', 'enhanced', 'dexterous', 'steadfast', 'noteworthy', 'advantageously', 'invincibility', 'exalted', 'assuring', 'rapturously', 'lawful', '

In [ ]:
final_positive = positive_words - stopword_set
final_negative = negative_words - stopword_set
print(len(final_positive), len(final_negative), sep="\n")



1906
4693


In [9]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
# from nltk.tokenize import RegexpTokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\monil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\monil\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\monil\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
from nltk.corpus import stopwords
import string
import re

In [ ]:
filter_file = os.listdir(filtered_article_folder)

def positive_filter(words):
    positive_count = 0
    for word in words:
        word = word.lower()
        if word in final_positive:
            positive_count+=1
    return positive_count

def negative_filter(words):
    negative_count = 0
    for word in words:
        word = word.lower()
        if word in final_negative:
            negative_count -=1
    return (negative_count * -1)

def polarity_filter(ps,ns):
    polarity_value = (ps-ns)/((ps+ns)+0.000001)
    return polarity_value

def subjectivity_filter(ps,ns):
    subjectivity_value = (ps+ns)/((len(word_lst))+0.000001)
    return subjectivity_value

def count_syllable(word):
    word = re.sub(r'(es|ed)$', '', word)
    syllables = len(re.findall(r'[aeiouy]+', word))
    return max(1, syllables)

def complex_word(filtered_after_punctuation):
    complex_lst = []
    for word in filtered_after_punctuation:
        word = word.lower()
        if count_syllable(word) > 2:
            complex_lst.append(word)
    return complex_lst

def calculate_pronoun(filtered_after_punctuation):
    count =0
    pronoun = ['i','we','ours','my','us']
    for word in filtered_after_punctuation:
        if(word == "US"):
            continue
        elif(word.lower() in pronoun):
            count+=1
    return count
    
def calculate_word_length(filtered_after_punctuation):

    total_character = sum(len(word) for word in filtered_after_punctuation)
    total_word  = len(filtered_after_punctuation)

    return total_character/total_word

result_lst = []
for file in filter_file:
    with open(f"{filtered_article_folder}\\{file}", "r", encoding ="utf-8") as f:
        content = f.read()
    
    word_lst = nltk.tokenize.word_tokenize(content)
    sentence_lst = nltk.tokenize.sent_tokenize(content)

    positive_score = positive_filter(word_lst)
    negative_score = negative_filter(word_lst)
    polarity_score = polarity_filter(positive_score,negative_score)
    subjectivity_score = subjectivity_filter(positive_score,negative_score)
  

    filtered_after_stopwords = [ word for word in word_lst if word.lower() not in stopwords.words('english') ] # words after removing stopwords
    filtered_after_punctuation = [ word for word in filtered_after_stopwords if word.lower() not in string.punctuation] # words after removing punctuation
   
    average_sent_length = len(filtered_after_punctuation)/len(sentence_lst)
    complex_word_count = len(complex_word(filtered_after_punctuation))

    syllable_count = 0
    for word in filtered_after_punctuation:
        word= word.lower()
        syllable_count += count_syllable(word)
    
    syllable_per_word = syllable_count/len(filtered_after_punctuation)
    complex_percentage = complex_word_count / len(filtered_after_punctuation)
    fog_index = 0.4 * (average_sent_length+complex_percentage)
    avg_word_per_sentence = len(word_lst)/len(sentence_lst) # Average number of words before cleaning
    pronoun_count = calculate_pronoun(filtered_after_punctuation)
    average_word_length = calculate_word_length(filtered_after_punctuation)
    # print(filtered_after_punctuation)


    result_lst.append([positive_score, negative_score, polarity_score, subjectivity_score, average_sent_length, complex_percentage, fog_index, \
          avg_word_per_sentence, complex_word_count, len(filtered_after_punctuation), syllable_per_word, pronoun_count, average_word_length ])


# for lst in result_lst:
#     print(lst)

[5, 1, 0.6666665555555741, 0.020338982981901753, 34.0, 0.3333333333333333, 13.733333333333334, 49.166666666666664, 68, 204, 2.215686274509804, 2, 7.294117647058823]
[10, 8, 0.11111110493827195, 0.018556701011797214, 14.194444444444445, 0.29354207436399216, 5.795194607523375, 26.944444444444443, 150, 511, 2.156555772994129, 1, 6.448140900195694]
[11, 4, 0.46666663555555765, 0.031249999934895835, 28.0, 0.24107142857142858, 11.29642857142857, 40.0, 81, 336, 2.0833333333333335, 0, 6.4523809523809526]
[23, 12, 0.3142857053061227, 0.05303030294995409, 16.62962962962963, 0.48329621380846327, 6.845170337375237, 24.444444444444443, 217, 449, 2.5835189309576836, 0, 7.7616926503340755]
[9, 2, 0.636363578512402, 0.013630731085959441, 11.744186046511627, 0.35247524752475246, 4.838664517614553, 18.767441860465116, 178, 505, 2.3049504950495048, 0, 6.815841584158416]
[5, 1, 0.6666665555555741, 0.020338982981901753, 34.0, 0.3333333333333333, 13.733333333333334, 49.166666666666664, 68, 204, 2.2156862745

In [31]:
final_df = df.copy()

In [40]:
final_df['POSITIVE SCORE'] = [x[0] for x in result_lst]
final_df['NEGATIVE SCORE'] = [x[1] for x in result_lst]
final_df['POLARITY SCORE'] = [x[2] for x in result_lst]
final_df['SUBJECTIVITY SCORE'] = [x[3] for x in result_lst]
final_df['AVG SENTENCE LENGTH'] = [x[4] for x in result_lst]
final_df['PERCENTAGE OF COMPLEX WORDS'] = [x[5] for x in result_lst]
final_df['FOG INDEX'] = [x[6] for x in result_lst]
final_df['AVG NUMBER OF WORDS PER SENTENCE'] = [x[7] for x in result_lst] 
final_df['COMPLEX WORD COUNT'] = [x[8] for x in result_lst]
final_df['WORD COUNT'] = [x[9] for x in result_lst]
final_df['SYLLABLE PER WORD'] = [x[10] for x in result_lst]
final_df['PERSONAL PRONOUNS'] = [x[11] for x in result_lst]
final_df['AVG WORD LENGTH'] = [x[12] for x in result_lst]

In [41]:
final_df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH
0,Netclan20241017,https://insights.blackcoffer.com/ai-and-ml-bas...,5,1,0.666667,0.020339,34.000000,0.333333,13.733333,49.166667,68,204,2.215686,2,7.294118
1,Netclan20241018,https://insights.blackcoffer.com/enhancing-fro...,10,8,0.111111,0.018557,14.194444,0.293542,5.795195,26.944444,150,511,2.156556,1,6.448141
2,Netclan20241019,https://insights.blackcoffer.com/roas-dashboar...,11,4,0.466667,0.031250,28.000000,0.241071,11.296429,40.000000,81,336,2.083333,0,6.452381
3,Netclan20241020,https://insights.blackcoffer.com/efficient-pro...,23,12,0.314286,0.053030,16.629630,0.483296,6.845170,24.444444,217,449,2.583519,0,7.761693
4,Netclan20241021,https://insights.blackcoffer.com/development-o...,9,2,0.636364,0.013631,11.744186,0.352475,4.838665,18.767442,178,505,2.304950,0,6.815842
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,Netclan20241159,https://insights.blackcoffer.com/population-an...,12,9,0.142857,0.021739,18.542857,0.328197,7.548422,27.600000,213,649,2.183359,0,6.329738
143,Netclan20241160,https://insights.blackcoffer.com/google-lsa-ap...,20,17,0.081081,0.025239,14.246154,0.307775,5.821572,22.553846,285,926,2.206263,0,6.342333
144,Netclan20241161,https://insights.blackcoffer.com/healthcare-da...,7,10,-0.176471,0.039535,14.470588,0.223577,5.877666,25.294118,55,246,2.085366,3,6.280488
145,Netclan20241162,https://insights.blackcoffer.com/budget-sales-...,1,0,0.999999,0.006536,118.000000,0.110169,47.244068,153.000000,13,118,1.932203,0,5.686441


In [43]:
final_df.to_excel("FinalOutput.xlsx", index=False)